In [12]:
import requests
import pandas as pd
from io import StringIO
from datetime import datetime

In [13]:
file_url = "https://sanctionslistservice.ofac.treas.gov/api/PublicationPreview/exports/SDN.CSV"

response = requests.get(file_url)

# Check if the request was successful
if response.status_code == 200:
    csv_content = StringIO(response.text)
    df = pd.read_csv(csv_content, header=None)
    terrorist = df.iloc[:, [1, 2]].rename(columns={1: 'Name', 2: 'Type'})
    
else:
    print(f"Failed to retrieve the file. Status code: {response.status_code}")

In [14]:
terrorist_individual = terrorist[terrorist['Type'].str.lower()=='individual']
terrorist_names = terrorist_individual[['Name']]

In [15]:
terrorist_names = terrorist_names.reset_index(drop=True) 

In [16]:
split_names = terrorist_names['Name'].str.split(',', expand=True)
terrorist_names['PERSONAL.LASTNAME'] = split_names.iloc[:, 0].str.strip()
terrorist_names['PERSONAL.FIRSTNAME'] = split_names.iloc[:, 1].str.strip() if split_names.shape[1] > 1 else None


In [17]:
terrorist_names = terrorist_names.drop(columns=['Name'])

In [18]:
terrorist_names = terrorist_names[['PERSONAL.FIRSTNAME','PERSONAL.LASTNAME']]

In [19]:
current_date = this_week = datetime.now().strftime('%Y-%m-%d')

In [20]:
filename = f"OFAC_Update_{current_date}.csv"
terrorist_names.to_csv(filename,index=False)